In [3]:
import pandas as pd
import numpy as np

In [30]:
onre=pd.read_csv("R:\\portfoli_projects\\Online retailes\\online_retail.csv")
print(onre.info())
print(onre.isnull().sum())
onre.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Data Cleaning:

In [133]:
#Are there any missing or null values 
#Check for NaN values using df.isnull().sum() or isna().sum()
onre.isnull().sum()
dd=onre

# Check for missing values in critical columns
critical_columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID']

missing_values = dd[critical_columns].isna().sum()
print(missing_values)


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
UnitPrice           0
CustomerID     135080
dtype: int64


In [135]:
#drop rows with missing values.
# Drop rows with missing critical columns
#only for that specific coloum  droping
dd=dd.loc[dd['CustomerID'].notna(), :]
dd=dd.loc[dd['Description'].notna(),:]
dd.isnull().sum()
dd.describe()

#(OR)
# Drop rows with missing critical columns
dd.dropna(subset=['InvoiceNo', 'StockCode', 'Description', 'CustomerID'], inplace=True)

# For Quantity and UnitPrice, you can either drop or impute
dd.dropna(subset=['Quantity', 'UnitPrice'], inplace=True)  # Drop rows with missing Quantity or Price


# Check unique values in CustomerID to see if there are any unexpected values
print(dd['CustomerID'].unique())


[17850. 13047. 12583. ... 13298. 14569. 12713.]


In [117]:

# Convert to numeric, invalid entries will be turned into NaN
dd['Quantity'] = pd.to_numeric(dd['Quantity'], errors='coerce')
dd['UnitPrice'] = pd.to_numeric(dd['UnitPrice'], errors='coerce')

# Now, you can filter out NaN values (which were originally non-numeric entries)
dd = dd.dropna(subset=['Quantity', 'UnitPrice'])

dd.count()

 #Invalid Quantity and Price values 
 #Convert 'Quantity' and 'UnitPrice' to strings before matching
#dd[dd['Quantity'].astype(str).str.match(r'^[^\d]+$', na=False)]  # For non-digit values in Quantity
#dd[dd['UnitPrice'].astype(str).str.match(r'^[^\d]+$', na=False)]  # For non-digit values in UnitPrice




InvoiceNo      397884
StockCode      397884
Description    397884
Quantity       397884
InvoiceDate    397884
UnitPrice      397884
CustomerID     397884
Country        397884
TotalSales     397884
dtype: int64

In [125]:
#Are there any duplicate rows
dd.duplicated()
# Remove any duplicate rows
dd.drop_duplicates()

# Check for duplicates based on InvoiceNo and StockCode
duplicates = dd[dd.duplicated(subset=['InvoiceNo', 'StockCode'], keep=False)]
print(duplicates)

# Remove duplicates
dd.drop_duplicates(subset=['InvoiceNo', 'StockCode'], keep='first', inplace=True)


Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, TotalSales]
Index: []


InvoiceNo      387841
StockCode      387841
Description    387841
Quantity       387841
InvoiceDate    387841
UnitPrice      387841
CustomerID     387841
Country        387841
TotalSales     387841
dtype: int64

In [127]:
# Replace missing values with "Unknown" for categorical columns
dd['Description'] = dd['Description'].fillna('Unknown')
dd['CustomerID'] = dd['CustomerID'].fillna('Unknown')
dd['Country'] = dd['Country'].fillna('Unknown')


In [151]:
# Define reasonable thresholds for Quantity and UnitPrice
quantity_threshold = 1000  # Example threshold for Quantity
price_threshold = 1000  # Example threshold for UnitPrice

# Filter out extreme outliers (adjust thresholds as needed)
dd = dd[(dd['Quantity'] <= quantity_threshold) & (dd['Quantity'] > 0)]
dd = dd[(dd['UnitPrice'] <= price_threshold) & (dd['UnitPrice'] > 0)]
print(dd.count())



InvoiceNo      397758
StockCode      397758
Description    397758
Quantity       397758
InvoiceDate    397758
UnitPrice      397758
CustomerID     397758
Country        397758
TotalSales     397758
dtype: int64


Exploratory Data Analysis (EDA):


In [82]:
#What is the total sales volume across all transactions
dd.loc[:, 'TotalSales'] = dd['Quantity'] * dd['UnitPrice'] #This explicitly tells pandas to modify the TotalSales column for all rows (the : part represents all rows).
tot=sum(dd['TotalSales'])
tot

9769872.054

In [147]:
#How does sales break down by country or customer segment
# First, create a new column to calculate the total sales for each row
dd.loc[:, 'TotalSales'] = dd['UnitPrice'] * dd['Quantity']

# Now group by 'Country' and sum up the sales
sales_by_country = dd.groupby('Country')['TotalSales'].sum().reset_index()

# Sort the result by TotalSales in descending order to see the countries with the highest sales
sales_by_country_sorted = sales_by_country.sort_values(by='TotalSales', ascending=False)

print(sales_by_country_sorted)


                 Country   TotalSales
35        United Kingdom  6902020.704
23           Netherlands   275523.780
10                  EIRE   261941.160
14               Germany   228867.140
13                France   199565.630
0              Australia   137887.710
30                 Spain    61577.110
32           Switzerland    56443.950
3                Belgium    41196.340
31                Sweden    38378.330
24                Norway    36165.440
26              Portugal    30955.930
19                 Japan    29970.370
12               Finland    22546.080
6        Channel Islands    20450.440
9                Denmark    18955.340
18                 Italy    17483.240
7                 Cyprus    13590.380
1                Austria    10198.680
29             Singapore     9522.290
25                Poland     7334.650
17                Israel     7221.690
15                Greece     4760.520
16               Iceland     4310.000
5                 Canada     3666.380
33          

In [153]:
print(dd.count())
dd

InvoiceNo      397758
StockCode      397758
Description    397758
Quantity       397758
InvoiceDate    397758
UnitPrice      397758
CustomerID     397758
Country        397758
TotalSales     397758
dtype: int64


In [149]:
# Save as CSV
dd.to_csv("Data_cleaning_on_online _retailer.csv", index=False)

# Save as Pickle (faster and preserves data types)
dd.to_pickle("Data_cleaning_on_online _retailer.pkl")
